In [ ]:
# importing necessary libraries
try:
  import dgl
except:
  !pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu121/repo.html
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
import numpy as np
from sklearn.model_selection import train_test_split

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 695.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Mounted at /content/drive


In [ ]:
# Download the jknet from our github :)
!wget https://raw.githubusercontent.com/Kirdon6/ATDL_assignment2/refs/heads/main/jknet.py
!wget https://raw.githubusercontent.com/Kirdon6/ATDL_assignment2/refs/heads/main/gat.py
!wget https://raw.githubusercontent.com/Kirdon6/ATDL_assignment2/refs/heads/main/gcn.py

In [ ]:
import jknet
import gat
import gcn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# SETUP Hyperparameters

In [ ]:
layers = range(1,7)
adam_lr = 5e-3
l2_reg = 5e-4
dropout = 0.5
hidden_dim = [16,32]
runs = 3

# Not stated in paper
epochs = 200

# Prepare Dataset

In [ ]:
def prepare_dataset(dataset):
    transform = (dgl.AddSelfLoop())
    if dataset == "Cora":
        dataset = dgl.data.CoraGraphDataset(transform=transform)
    elif dataset == "Citeseer":
        dataset = dgl.data.CiteseerGraphDataset(transform=transform)
    else:
      raise ValueError("Dataset {} is invalid.".format(dataset))
    graph = dataset[0]
    graph = graph.to(device)

    n_classes = dataset.num_classes

    labels = graph.ndata.pop("label").to(device).long()

    features = graph.ndata.pop("feat").to(device)
    n_features = features.shape[-1]


    n_nodes = graph.num_nodes()
    idx = torch.arange(n_nodes).to(device)
    train_idx, test_idx = train_test_split(idx, test_size=0.2)
    train_idx, val_idx = train_test_split(train_idx, test_size=0.25)


    return graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features

# Essential Functions

In [ ]:
def train(graph, features, labels, train_idx,epochs, model, loss_fn, optimizer):
    # define train/val samples, loss function and optimizer
    print("Training...")
    # training loop
    for epoch in range(epochs):
        # print(f"Epoch {epoch}:")
        model.train()
        logits = model(graph, features)
        train_loss = loss_fn(logits[train_idx], labels[train_idx])
        train_acc = torch.sum(
            logits[train_idx].argmax(dim=1) == labels[train_idx]
        ).item() / len(train_idx)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # print(f"Loss: {train_loss}")
        # print(f"Accuracy: {train_acc}")

def evaluate(graph, features, labels, val_idx, model, loss_fn):
    print("Evaluating on val subset...")
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        valid_loss = loss_fn(logits[val_idx], labels[val_idx])
        valid_acc = torch.sum(
            logits[val_idx].argmax(dim=1) == labels[val_idx]
        ).item() / len(val_idx)

    # Print out performance
    print(f"Validation Loss: {valid_loss}")
    print(f"Validation Accuracy: {valid_acc}")
    print("")


def test(graph, features, labels, test_idx, model):
    print("Testing...")
    model.eval()
    logits = model(graph, features)
    test_acc = torch.sum(
      logits[test_idx].argmax(dim=1) == labels[test_idx]
    ).item() / len(test_idx)

    print(f"Test Accuracy: {test_acc}")
    return test_acc

# Cora dataset

In [ ]:
graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features = prepare_dataset("Cora")

/root/.dgl/cora_v2.zip:   0%|          | 0.00/132k [00:00<?, ?B/s]

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


# JKNET

## JKNet Concat

### Training

In [ ]:
# Training and tuning JKNet_concat
for dim in hidden_dim:
  for num_layers in layers:
    print(f"Training JKNet-cat with dim={dim} and layers={num_layers}")
    print("")
    JKNet = jknet.JKNet(in_dim=n_features, hid_dim=dim,  out_dim=n_classes, num_layers=num_layers, mode='cat', dropout=dropout).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(JKNet.parameters(), lr=adam_lr, weight_decay=l2_reg)
    train(graph, features, labels, train_idx, epochs, JKNet, loss_fn, optimizer)
    evaluate(graph, features, labels, val_idx, JKNet, loss_fn)

Training JKNet-cat with dim=16 and layers=1

Training...
Evaluating on val subset...
Validation Loss: 0.42941051721572876
Validation Accuracy: 0.8819188191881919

Training JKNet-cat with dim=16 and layers=2

Training...
Evaluating on val subset...
Validation Loss: 0.46647900342941284
Validation Accuracy: 0.8671586715867159

Training JKNet-cat with dim=16 and layers=3

Training...
Evaluating on val subset...
Validation Loss: 0.48921433091163635
Validation Accuracy: 0.8726937269372693

Training JKNet-cat with dim=16 and layers=4

Training...
Evaluating on val subset...
Validation Loss: 0.4783461391925812
Validation Accuracy: 0.8708487084870848

Training JKNet-cat with dim=16 and layers=5

Training...
Evaluating on val subset...
Validation Loss: 0.5445578098297119
Validation Accuracy: 0.8560885608856088

Training JKNet-cat with dim=16 and layers=6

Training...
Evaluating on val subset...
Validation Loss: 0.529123067855835
Validation Accuracy: 0.8671586715867159

Training JKNet-cat with di

## JKNet Max

### Training

In [ ]:
for dim in hidden_dim:
  for num_layers in layers:
    print(f"Training JKNet-max with dim={dim} and layers={num_layers}")
    JKNet = jknet.JKNet(in_dim=n_features, hid_dim=dim,  out_dim=n_classes, num_layers=num_layers, mode='max', dropout=dropout).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(JKNet.parameters(), lr=adam_lr, weight_decay=l2_reg)
    train(graph, features, labels, train_idx, epochs, JKNet, loss_fn, optimizer)
    evaluate(graph, features, labels, val_idx, JKNet, loss_fn)

Training JKNet-max with dim=16 and layers=1
Training...
Evaluating on val subset...
Validation Loss: 0.4986284673213959
Validation Accuracy: 0.8726937269372693

Training JKNet-max with dim=16 and layers=2
Training...
Evaluating on val subset...
Validation Loss: 0.5000510811805725
Validation Accuracy: 0.8763837638376384

Training JKNet-max with dim=16 and layers=3
Training...
Evaluating on val subset...
Validation Loss: 0.5048632621765137
Validation Accuracy: 0.8782287822878229

Training JKNet-max with dim=16 and layers=4
Training...
Evaluating on val subset...
Validation Loss: 0.49434995651245117
Validation Accuracy: 0.8745387453874539

Training JKNet-max with dim=16 and layers=5
Training...
Evaluating on val subset...
Validation Loss: 0.5114761590957642
Validation Accuracy: 0.8671586715867159

Training JKNet-max with dim=16 and layers=6
Training...
Evaluating on val subset...
Validation Loss: 0.5034537315368652
Validation Accuracy: 0.8745387453874539

Training JKNet-max with dim=32 an

## JKNet LSTM

### Training

In [ ]:
for dim in hidden_dim:
  for num_layers in layers:
    print(f"Training JKNet-lstm with dim={dim} and layers={num_layers}")
    JKNet = jknet.JKNet(in_dim=n_features, hid_dim=dim,  out_dim=n_classes, num_layers=num_layers, mode='lstm', dropout=dropout).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(JKNet.parameters(), lr=adam_lr, weight_decay=l2_reg)
    train(graph, features, labels, train_idx, epochs, JKNet, loss_fn, optimizer)
    evaluate(graph, features, labels, val_idx, JKNet, loss_fn)

Training JKNet-lstm with dim=16 and layers=1
Training...
Evaluating on val subset...
Validation Loss: 0.4836644232273102
Validation Accuracy: 0.8782287822878229

Training JKNet-lstm with dim=16 and layers=2
Training...
Evaluating on val subset...
Validation Loss: 0.5232759118080139
Validation Accuracy: 0.8782287822878229

Training JKNet-lstm with dim=16 and layers=3
Training...
Evaluating on val subset...
Validation Loss: 0.5379359126091003
Validation Accuracy: 0.8653136531365314

Training JKNet-lstm with dim=16 and layers=4
Training...
Evaluating on val subset...
Validation Loss: 0.4980649948120117
Validation Accuracy: 0.8763837638376384

Training JKNet-lstm with dim=16 and layers=5
Training...
Evaluating on val subset...
Validation Loss: 0.5144003629684448
Validation Accuracy: 0.8782287822878229

Training JKNet-lstm with dim=16 and layers=6
Training...
Evaluating on val subset...
Validation Loss: 0.4906941056251526
Validation Accuracy: 0.8819188191881919

Training JKNet-lstm with dim

# GCN

## Training

In [ ]:
for dim in hidden_dim:
  for num_layers in layers:
    print(f"Training GCN with dim={dim} and layers={num_layers}")
    GCN = gcn.GCN(in_size=n_features, hid_size=dim, out_size=n_classes, num_layers=num_layers, dropout=dropout).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(GCN.parameters(), lr=adam_lr, weight_decay=l2_reg)
    train(graph, features, labels, train_idx, epochs, GCN, loss_fn, optimizer)
    evaluate(graph, features, labels, val_idx, GCN, loss_fn)

Training GCN with dim=16 and layers=1
Training...
Evaluating on val subset...
Validation Loss: 0.5463233590126038
Validation Accuracy: 0.8560885608856088

Training GCN with dim=16 and layers=2
Training...
Evaluating on val subset...
Validation Loss: 0.5518293380737305
Validation Accuracy: 0.8523985239852399

Training GCN with dim=16 and layers=3
Training...
Evaluating on val subset...
Validation Loss: 0.8348432779312134
Validation Accuracy: 0.7306273062730627

Training GCN with dim=16 and layers=4
Training...
Evaluating on val subset...
Validation Loss: 0.9503019452095032
Validation Accuracy: 0.6512915129151291

Training GCN with dim=16 and layers=5
Training...
Evaluating on val subset...
Validation Loss: 1.002485990524292
Validation Accuracy: 0.6217712177121771

Training GCN with dim=16 and layers=6
Training...
Evaluating on val subset...
Validation Loss: 1.145426869392395
Validation Accuracy: 0.6217712177121771

Training GCN with dim=32 and layers=1
Training...
Evaluating on val subs

# GAT

### Training

In [ ]:
for dim in hidden_dim:
  for num_layers in layers:
    print(f"Training GAT with dim={dim} and layers={num_layers}")
    GAT = gat.GAT(in_size=n_features, hid_size=dim, out_size=n_classes, heads=[dim, 1], num_layers=num_layers).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(GAT.parameters(), lr=adam_lr, weight_decay=l2_reg)
    train(graph, features, labels, train_idx, epochs, GAT, loss_fn, optimizer)
    evaluate(graph, features, labels, val_idx, GAT, loss_fn)

Training GAT with dim=16 and layers=1
Training...
Evaluating on val subset...
Validation Loss: 0.3763570487499237
Validation Accuracy: 0.8763837638376384

Training GAT with dim=16 and layers=2
Training...
Evaluating on val subset...
Validation Loss: 0.3886018395423889
Validation Accuracy: 0.8634686346863468

Training GAT with dim=16 and layers=3
Training...
Evaluating on val subset...
Validation Loss: 0.42799580097198486
Validation Accuracy: 0.8671586715867159

Training GAT with dim=16 and layers=4
Training...
Evaluating on val subset...
Validation Loss: 0.38184937834739685
Validation Accuracy: 0.8819188191881919

Training GAT with dim=16 and layers=5
Training...
Evaluating on val subset...
Validation Loss: 0.5085490942001343
Validation Accuracy: 0.8634686346863468

Training GAT with dim=16 and layers=6
Training...
Evaluating on val subset...
Validation Loss: 0.47972676157951355
Validation Accuracy: 0.8616236162361623

Training GAT with dim=32 and layers=1
Training...
Evaluating on val

## Testing Cora

Note that standard deviations in this code are popluation standard deviations.
In the report, we provide sample standard deviations instead.
Code for the sample standard deviation would be:<br>
`std = np.around(np.std(acc_list, axis=0, ddof=1), decimals=3)`

In [ ]:
# Evaluate best model
acc_list = list()
for _ in range(runs):
  # always create new splits
  graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features = prepare_dataset("Cora")
  JKNet = jknet.JKNet(in_dim=n_features, hid_dim=16,  out_dim=n_classes, num_layers=1, mode='cat', dropout=dropout).to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(JKNet.parameters(), lr=adam_lr, weight_decay=l2_reg)
  train(graph, features, labels, train_idx, epochs, JKNet, loss_fn, optimizer)
  acc = test(graph, features, labels, test_idx, JKNet)
  acc_list.append(acc)

mean = np.around(np.mean(acc_list, axis=0), decimals=3)
std = np.around(np.std(acc_list, axis=0), decimals=3)
print("Total acc:", acc_list)
print("Mean:", mean)
print("Std:", std)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8911439114391144
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8856088560885609
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8911439114391144
Total acc: [0.8911439114391144, 0.8856088560885609, 0.8911439114391144]
Mean: 0.889
Std: 0.003


In [ ]:
# Evaluate best model
acc_list = list()
for _ in range(runs):
  graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features = prepare_dataset("Cora")
  JKNet = jknet.JKNet(in_dim=n_features, hid_dim=16,  out_dim=n_classes, num_layers=3, mode='max', dropout=dropout).to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(JKNet.parameters(), lr=adam_lr, weight_decay=l2_reg)
  train(graph, features, labels, train_idx, epochs, JKNet, loss_fn, optimizer)
  acc = test(graph, features, labels, test_idx, JKNet)
  acc_list.append(acc)

mean = np.around(np.mean(acc_list, axis=0), decimals=3)
std = np.around(np.std(acc_list, axis=0), decimals=3)
print("Total acc: ", acc_list)
print("Mean", mean)
print("Std", std)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8634686346863468
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8726937269372693
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8542435424354243
Total acc:  [0.8634686346863468, 0.8726937269372693, 0.8542435424354243]
Mean 0.863
Std 0.008


In [ ]:
# Evaluate best model
acc_list = list()
for _ in range(runs):
  graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features = prepare_dataset("Cora")
  JKNet = jknet.JKNet(in_dim=n_features, hid_dim=16,  out_dim=n_classes, num_layers=6, mode='lstm', dropout=dropout).to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(JKNet.parameters(), lr=adam_lr, weight_decay=l2_reg)
  train(graph, features, labels, train_idx, epochs, JKNet, loss_fn, optimizer)
  acc = test(graph, features, labels, test_idx, JKNet)
  acc_list.append(acc)

mean = np.around(np.mean(acc_list, axis=0), decimals=3)
std = np.around(np.std(acc_list, axis=0), decimals=3)
print("Total acc: ", acc_list)
print("Mean", mean)
print("Std", std)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8044280442804428
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8874538745387454
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8726937269372693
Total acc:  [0.8044280442804428, 0.8874538745387454, 0.8726937269372693]
Mean 0.855
Std 0.036


In [ ]:
acc_list = list()
for _ in range(runs):
  graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features = prepare_dataset("Cora")
  GCN = gcn.GCN(in_size=n_features, hid_size=32, out_size=n_classes, num_layers=1, dropout=dropout).to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(GCN.parameters(), lr=adam_lr, weight_decay=l2_reg)
  train(graph, features, labels, train_idx, epochs, GCN, loss_fn, optimizer)
  acc = test(graph, features, labels, test_idx, GCN)
  acc_list.append(acc)

mean = np.around(np.mean(acc_list, axis=0), decimals=3)
std = np.around(np.std(acc_list, axis=0), decimals=3)
print("Total acc: ", acc_list)
print("Mean", mean)
print("Std", std)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.9040590405904059
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8671586715867159
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8782287822878229
Total acc:  [0.9040590405904059, 0.8671586715867159, 0.8782287822878229]
Mean 0.883
Std 0.015


In [ ]:
acc_list = list()
for _ in range(runs):
  graph, features, labels, train_idx, val_idx, test_idx, n_classes, n_features = prepare_dataset("Cora")
  GAT = gat.GAT(in_size=n_features, hid_size=16, out_size=n_classes, heads=[16, 1], num_layers=4).to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(GAT.parameters(), lr=adam_lr, weight_decay=l2_reg)
  train(graph, features, labels, train_idx, epochs, GAT, loss_fn, optimizer)
  acc = test(graph, features, labels, test_idx, GAT)
  acc_list.append(acc)

mean = np.around(np.mean(acc_list, axis=0), decimals=3)
std = np.around(np.std(acc_list, axis=0), decimals=3)
print("Total acc: ", acc_list)
print("Mean", mean)
print("Std", std)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8726937269372693
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8856088560885609
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Testing...
Test Accuracy: 0.8597785977859779
Total acc:  [0.8726937269372693, 0.8856088560885609, 0.8597785977859779]
Mean 0.873
Std 0.011
